# Загрузка данных

Данные можно найти по [ссылке](https://drive.google.com/drive/folders/1GiL4FCOYVBB61wcCjN_qTHY18wkOMrRk?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import csv


df = pd.read_csv("/content/drive/MyDrive/data_drive2/drive2_BMW.csv", delimiter=';', on_bad_lines='skip', quotechar="'", engine='python')

In [ ]:
rows_to_join = []

to_add = 0
flag = False
for i in range(len(df)):
    if pd.isna(df.at[i, "Время публикации"]) and pd.isna(df.at[i, "Описание"]):
      if not flag:
        to_add = i - 1
        flag = True
      rows_to_join.append(df.at[i, "Модель автомобиля"])
      df.drop(i, inplace=True)
    else:
      if rows_to_join:
        if pd.isna(df.at[to_add, "Описание"]):
          df.at[to_add, "Описание"] = ""
        df.at[to_add, "Описание"] += " " + " ".join(rows_to_join)
        rows_to_join = []
        flag = False

df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(20)

,Модель автомобиля,Время публикации,Описание
0,BMW X1,2025-01-17,Добрый день. Листал как то Яндекс маркет и нат...
1,BMW 3 series AventurineRot «Вишенка»,2025-01-17,"""Так, ну я, в общем, всего месяц не думал о ма..."
2,BMW X5 40d,2025-01-17,"Пришла посылка, пока думаем, что с ней делать"
3,BMW 5 series Experimental,2025-01-17,"""Нашел человека который сделает pappel natur П..."
4,BMW 5 series LifestyleEdition 3.0d MT,2025-01-17,"""Народ ✌️ всем привет, с прошедшими всех празд..."
5,BMW 3 series Красная машинка жены,2025-01-17,"""Продолжаю приваривать морду, много мелочей вы..."
6,BMW X5 RHINO 40d,2025-01-17,"""Всем привет ! Заранее хочу поблагодарить, кто..."
7,BMW X5,2025-01-17,"""Господа, пара вопросов: Что за синий разьем в..."
8,BMW 7 series 🇩🇪 tiefe Nacht 🇩🇪,2025-01-17,"""Всем привет👋 Запись больше для себя и истории..."
9,BMW 7 series Баржа,2025-01-17,"""Бумер-тормоз все еще не ездит. Первая часть з..."


In [ ]:
df = df.dropna()

# Предобработка текстовых данных

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy


nlp = spacy.load("ru_core_news_sm")

In [ ]:
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.7 MB/s eta 0:00:00


In [ ]:
import demoji

def remove_emoji(text):
    new_text = demoji.replace(text, '')
    return new_text

In [ ]:
def preprocess(text):
  text = remove_emoji(text)
  doc = nlp(' '.join(list(map(lambda x: x.lower(), text.split()))))
  no_punct = [token for token in doc if not token.is_punct]
  no_stop = [token for token in no_punct if not token.is_stop]
  return " ".join([token.lemma_ for token in no_stop])

In [ ]:
df["Предобработанные данные"] = df["Описание"].apply(preprocess)
df

,Модель автомобиля,Время публикации,Описание,Предобработанные данные
0,BMW X1,2025-01-17,Добрый день. Листал как то Яндекс маркет и нат...,добрый день листать яндекс маркет наткнуться к...
1,BMW 3 series AventurineRot «Вишенка»,2025-01-17,"""Так, ну я, в общем, всего месяц не думал о ма...",общий месяц думать машина приехать проведать п...
2,BMW X5 40d,2025-01-17,"Пришла посылка, пока думаем, что с ней делать",прийти посылка думать делать
3,BMW 5 series Experimental,2025-01-17,"""Нашел человека который сделает pappel natur П...",найти человек сделать pappel natur поехать тар...
4,BMW 5 series LifestyleEdition 3.0d MT,2025-01-17,"""Народ ✌️ всем привет, с прошедшими всех празд...",народ привет прошедшими праздниками вопросик п...
...,...,...,...,...
5917,BMW 5 series Бестия,2024-12-17,"""Сборка затянулась 😵‍💫 ну радует только то что...",сборка затянуться радовать финишный прямая ост...
5918,BMW X5 ХАпять,2024-12-17,"""Всем привет !))) Немного полезной информации,...",привет полезный информация причина последствие...
5921,BMW X5,2024-12-17,"""Имеется новый нерабочий ДМРВ vdo.Несовсем кон...",иметься новый нерабочий дмрв vdo.несовсем новы...
5922,BMW 4 series ❄440xdrive,2024-12-17,"""Всем привет, давно как мне казалось для внешн...",привет внешний вид 440 хватать порог губа коле...


In [ ]:
def preprocess2(text):
  text = ' '.join([i for i in text.split() if not i.isdigit()])
  return text

df["Предобработанные данные"] = df["Предобработанные данные"].apply(preprocess2)

# Bag-Of-Words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
bow = vectorizer.fit_transform([df['Предобработанные данные'][0]])
count_array = bow.toarray()
features = vectorizer.get_feature_names_out()

df_bow = pd.DataFrame(data=count_array, columns=features)

Получили частоту встречаемости каждого слова в отдельно взятом сообщении.

In [ ]:
df_bow.transpose()

,0
алик,1
белый,1
бмв,1
взять,1
вч,2
выход,1
гу,1
дело,1
день,2
добрый,1


Посмотрим на топ-10 наиболее часто встречающихся слов:

In [ ]:
df_bow.iloc[0].nlargest(10).index.tolist()

['вч',
 'день',
 'заказать',
 'искать',
 'колонка',
 'комплект',
 'черно',
 'яндекс',
 'алик',
 'белый']

В целом, этого уже достаточно, чтобы сделать минимальные выводы об изначальном тексте: проблемы никакой у автовладельца нет, он просто решил обновить акустическую систему в своем автомобиле.

# TF-IDF model

В данном алгоритме значимость отдельного слова в каждом сообщении оценивается относительно встречаемости этого слова во всех сообщениях датасета.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer2 = TfidfVectorizer()
tf_idf = vectorizer2.fit_transform(df['Предобработанные данные'])
count_array = tf_idf.toarray()
features = vectorizer2.get_feature_names_out()

df_tf_idf = pd.DataFrame(data=count_array, columns=features)

In [ ]:
df_tf_idf.iloc[0].nlargest(10).index.tolist()

['черно',
 'вч',
 'яндекс',
 'колонка',
 'подпитать',
 'подцепиться',
 'жгуте',
 'рубеж',
 'маркет',
 'искать']

Сравнивая этот список с полученным выше, видим:
1. Некоторые слова встречаются одновременно в обоих списках двух рассматриваемых алгоритмов.
2. Алгоритм TF-IDF выделил слова, характерные для тематики автомобильного форума, в отличие от BOW.